<a href="https://colab.research.google.com/github/hoky1227/bert_based-recommendation/blob/main/Transformer_Pre_processing3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
# !pip install pytorch-transformers

In [3]:
import tensorflow as tf
import torch
# import tensorflow_datasets as tfds

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# import numpy as np
# import pandas as pd
# import torch
from torch.utils.data import Dataset
# from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

import pandas as pd
import numpy as np
import random
import time
import datetime
import csv
import os
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("/content/drive/MyDrive/2017online retail.csv")
df.head()
# df.describe()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Customer,Payment Way,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,Skrill,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,Skrill,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,Skrill,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,Skrill,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,Credit,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49


In [5]:
# Detail = pd.read_csv('/content/drive/MyDrive/Detail.csv')
# Detail.pop('Unnamed: 0')
# Detail.head()

In [6]:
# detail = Detail['0'].unique()
# detail = detail.tolist()
# len(detail)

In [7]:
# dic = {string : i for i,string in enumerate(detail)}
# print(dic)
# len(dic)

In [8]:
print(df.isnull().sum())
df.dropna(inplace=True)
df.isnull().sum()

Customer       42970
Payment Way        0
Invoice            0
ItemCode           0
Detail          1745
Date               0
Amount             0
Price              0
dtype: int64


Customer       0
Payment Way    0
Invoice        0
ItemCode       0
Detail         0
Date           0
Amount         0
Price          0
dtype: int64

In [9]:
df

,Customer,Payment Way,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,Skrill,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,Skrill,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,Skrill,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,Skrill,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,Credit,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49
...,...,...,...,...,...,...,...,...
210359,34590.0,Credit,243414,21527,RETRO SPOT TRADITIONAL TEAPOT,2017/1/4 10:28,12,6.95
210360,34590.0,Skrill,243414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2017/1/4 10:28,2,2.55
210361,34590.0,Credit,243414,35001G,HAND OPEN SHAPE GOLD,2017/1/4 10:28,2,4.25
210365,32346.0,Credit,243412,TEST001,This is a test product.,2017/1/4 9:53,5,4.50


In [10]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
df.head()

1458


,Customer,Payment Way,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,Skrill,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,Skrill,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,Skrill,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,Skrill,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,Credit,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49


In [11]:
df.drop(['Payment Way'],axis=1, inplace=True)
df.head()

,Customer,Invoice,ItemCode,Detail,Date,Amount,Price
0,37593.0,C514216,22406,MONEY BOX KINGS CHOICE DESIGN,2017/6/30 17:38,-1,1.25
1,37593.0,C514216,22056,CERAMIC PLATE LOVE HEART DESIGN,2017/6/30 17:38,-1,1.49
2,37593.0,C514216,21844,RED RETROSPOT MUG,2017/6/30 17:38,-1,2.95
3,37593.0,C514216,22073,RED RETROSPOT STORAGE JAR,2017/6/30 17:38,-1,3.75
4,37593.0,C514216,22059,CERAMIC STRWBERRY DESIGN MUG,2017/6/30 17:38,-2,1.49


In [12]:
df['Detail'] = df['Detail'].str.strip()
Detail = df['Detail'].unique()
for i in Detail:
    if i.isupper() == False:
        print(i)

Manual
BAG 250g SWIRLY MARBLES
Discount
Adjustment by Peter on 24/05/2010 1
BAG 125g SWIRLY MARBLES
POLYESTER FILLER PAD 65CMx65CM
Bank Charges
POLYESTER FILLER PAD 40x40cm
Adjustment by john on 26/01/2010 17
Adjustment by john on 26/01/2010 16
Adjust bad debt
FOLK ART GREETING CARD,pack/12
POLYESTER FILLER PAD 45x45cm
Adjustment by Peter on Jun 25 2010
ESSENTIAL BALM 3.5g TIN IN ENVELOPE
SET Of 6 SOLDIER SKITTLES
POLYESTER FILLER PAD 30CMx30CM
THE KING GIFT BAG 25x24x12cm
POLYESTER FILLER PAD 45x30cm
This is a test product.
POLYESTER FILLER PAD 60x40cm


In [13]:
idx = df[df['Amount'] <= 0.].index
df.drop(idx, inplace=True)
l = ['Manual', 'This is a test product.', 'Discount', 'Adjustment by Peter on 24/05/2010 1',\
     'Bank Charges', 'Adjustment by john on 26/01/2010 17',  'Adjustment by john on 26/01/2010 16', 'Adjust bad debt', 'Adjustment by Peter on Jun 25 2010', 'POSTAGE', 'CARRIAGE']
for v in l:
    idx2 = df[df['Detail'] == v].index
    df.drop(idx2, inplace=True)
idx3 = df[df['Price'] <= 0.].index
df.drop(idx3, inplace=True)
print(df.head())
df.describe()

      Customer Invoice ItemCode  ...             Date Amount  Price
4492   34882.0  264212    84755  ...  2017/6/30 17:04     16   0.65
4493   34882.0  264212    20957  ...  2017/6/30 17:04     16   1.45
4494   34882.0  264212    22457  ...  2017/6/30 17:04     12   2.95
4496   34882.0  264212    20958  ...  2017/6/30 17:04      8   1.95
4497   34882.0  264212    22430  ...  2017/6/30 17:04      3   4.95

[5 rows x 7 columns]


,Customer,Amount,Price
count,161040.000000,161040.000000,161040.000000
mean,35334.207570,14.045293,3.051039
std,1673.922109,123.241911,3.435328
min,32346.000000,1.000000,0.001000
25%,33976.000000,2.000000,1.250000
50%,35248.000000,5.000000,1.950000
75%,36805.000000,12.000000,3.750000
max,38287.000000,19152.000000,125.000000


In [14]:
df['Detail'] = df['Detail'].str.strip()
Detail = df['Detail'].unique()
for i in Detail:
    if i.isupper() == False:
        print(i)

BAG 250g SWIRLY MARBLES
FOLK ART GREETING CARD,pack/12
POLYESTER FILLER PAD 45x45cm
POLYESTER FILLER PAD 40x40cm
BAG 125g SWIRLY MARBLES
POLYESTER FILLER PAD 65CMx65CM
ESSENTIAL BALM 3.5g TIN IN ENVELOPE
SET Of 6 SOLDIER SKITTLES
POLYESTER FILLER PAD 30CMx30CM
THE KING GIFT BAG 25x24x12cm
POLYESTER FILLER PAD 45x30cm
POLYESTER FILLER PAD 60x40cm


In [15]:
for i in Detail:
    if len(i) <= 10:
        print(i)

PHOTO CUBE
SOMBRERO
BINGO SET
SPACE OWL
CAT BOWL
SPOTS MUG
SPACE FROG
DAISY RING


In [16]:
df.drop(['Amount', 'Price', 'ItemCode'],axis=1, inplace=True)
df.head()

,Customer,Invoice,Detail,Date
4492,34882.0,264212,COLOUR GLASS T-LIGHT HOLDER HANGING,2017/6/30 17:04
4493,34882.0,264212,PORCELAIN HANGING BELL SMALL,2017/6/30 17:04
4494,34882.0,264212,NATURAL SLATE HEART CHALKBOARD,2017/6/30 17:04
4496,34882.0,264212,PORCELAIN HANGING BELL LARGE,2017/6/30 17:04
4497,34882.0,264212,ENAMEL WATERING CAN CREAM,2017/6/30 17:04


In [17]:
df['Invoice'] = df['Invoice'].apply(lambda x: str(x))

In [18]:
def sortdf(data, column):
    data.sort_values(column, inplace=True)
    data.reset_index(inplace=True)
    data.pop('index')
    return data

In [19]:
df = sortdf(df, ['Invoice'])
df.head()

,Customer,Invoice,Detail,Date
0,34590.0,243414,HAND OPEN SHAPE GOLD,2017/1/4 10:28
1,34590.0,243414,RETRO SPOT MUG,2017/1/4 10:28
2,34590.0,243414,RETRO SPOT SUGAR JAM BOWL,2017/1/4 10:28
3,34590.0,243414,NEW ENGLAND CERAMIC CAKE SERVER,2017/1/4 10:28
4,34590.0,243414,RETRO SPOT TRADITIONAL TEAPOT,2017/1/4 10:28


In [20]:
Date = df['Date'].unique()
Date = sorted(Date)
print(len(Date))

Invoice = df['Invoice'].unique()
Invoice = sorted(Invoice)
len(Invoice)

7178


7557

In [21]:
df.pop('Date')
df.head()

,Customer,Invoice,Detail
0,34590.0,243414,HAND OPEN SHAPE GOLD
1,34590.0,243414,RETRO SPOT MUG
2,34590.0,243414,RETRO SPOT SUGAR JAM BOWL
3,34590.0,243414,NEW ENGLAND CERAMIC CAKE SERVER
4,34590.0,243414,RETRO SPOT TRADITIONAL TEAPOT


In [22]:
df.pop('Customer')
df

,Invoice,Detail
0,243414,HAND OPEN SHAPE GOLD
1,243414,RETRO SPOT MUG
2,243414,RETRO SPOT SUGAR JAM BOWL
3,243414,NEW ENGLAND CERAMIC CAKE SERVER
4,243414,RETRO SPOT TRADITIONAL TEAPOT
...,...,...
161035,264212,ENAMEL WATERING CAN CREAM
161036,264212,PORCELAIN HANGING BELL LARGE
161037,264212,NATURAL SLATE HEART CHALKBOARD
161038,264212,PORCELAIN HANGING BELL SMALL


In [23]:
for v in Invoice:
    a = df[df['Invoice'] == v]
    if len(a) < 5:
        idx = a.index
        df.drop(idx, inplace=True)
df

,Invoice,Detail
0,243414,HAND OPEN SHAPE GOLD
1,243414,RETRO SPOT MUG
2,243414,RETRO SPOT SUGAR JAM BOWL
3,243414,NEW ENGLAND CERAMIC CAKE SERVER
4,243414,RETRO SPOT TRADITIONAL TEAPOT
...,...,...
161035,264212,ENAMEL WATERING CAN CREAM
161036,264212,PORCELAIN HANGING BELL LARGE
161037,264212,NATURAL SLATE HEART CHALKBOARD
161038,264212,PORCELAIN HANGING BELL SMALL


In [24]:
df2 = pd.DataFrame(columns=df.columns)
values = df['Invoice'].value_counts().index.sort_values()

for i, v in enumerate(values):
    df2.loc[i] = (v, '&&'.join(df[df['Invoice'] == v]['Detail']))
df2

,Invoice,Detail
0,243414,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,243427,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,243428,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,243432,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,243433,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...,...
6281,264208,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
6282,264209,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
6283,264210,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
6284,264211,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [25]:
df2.pop('Invoice')
df2

,Detail
0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...
1,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...
2,LUNCHBOX WITH CUTLERY FAIRY CAKES&&STRAWBERRY ...
3,RED WOOLLY HOTTIE WHITE HEART.&&CERAMIC CAKE D...
4,STRAWBERRY CERAMIC TRINKET BOX&&JUMBO BAG BAR...
...,...
6281,ASSORTED COLOUR BIRD ORNAMENT&&VICTORIAN GLASS...
6282,PACK OF 60 PINK PAISLEY CAKE CASES&&WASHROOM M...
6283,GLASS JAR DIGESTIVE BISCUITS&&BLUE STRIPE CERA...
6284,PICNIC BASKET WICKER SMALL&&NATURAL SLATE HEAR...


In [26]:
df_series = df2['Detail'].str.split('&&')
df_series[0]

['HAND OPEN SHAPE GOLD',
 'RETRO SPOT MUG',
 'RETRO SPOT SUGAR JAM BOWL',
 'NEW ENGLAND CERAMIC CAKE SERVER',
 'RETRO SPOT TRADITIONAL TEAPOT',
 'RETRO SPOT LARGE MILK JUG']

In [27]:
sequence_length = 5
step_size = 2

# [4, 4, 5, 5, 6, 6]이라면, 아래 함수를 통해
# [[4, 4, 5, 5], [5, 5, 6, 6]]으로 다시 만들어집니다.
def create_sequences(values, window_size, step_size):
    sequences = []
    start_index = 0
    while True:
        end_index = start_index + window_size
        seq = values[start_index:end_index]
        # 뒤에 남은 데이터가 window_size보다 작으면 중지합니다.
        if len(seq) < window_size:
            seq = values[-window_size:]
            # window_size와 같으면 list에 붙여줍니다.
            if len(seq) == window_size:
                sequences.append(seq)
            break
        sequences.append(seq)
        start_index += step_size
    return sequences


# ratings_data.movie_ids = ratings_data.movie_ids.apply(
#     lambda ids: create_sequences(ids, sequence_length, step_size)
# )

# ratings_data.ratings = ratings_data.ratings.apply(
#     lambda ids: create_sequences(ids, sequence_length, step_size)
# )

# del ratings_data["timestamps"]

In [28]:
for i, v in enumerate(df_series):
    df_series[i] = create_sequences(v, sequence_length, step_size)
df_series[0]

[['HAND OPEN SHAPE GOLD',
  'RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT'],
 ['RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT',
  'RETRO SPOT LARGE MILK JUG']]

In [29]:
df_list = []
for v in df_series:
    for value in v:
        df_list.append(value)
df_list[:2]

[['HAND OPEN SHAPE GOLD',
  'RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT'],
 ['RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT',
  'RETRO SPOT LARGE MILK JUG']]

In [30]:
label_list = []

for i in range(len(df_list)):
    l = df_list[i][-1]
    df_list[i].pop()
    label_list.append(l)

In [31]:
df_list[:2]

[['HAND OPEN SHAPE GOLD',
  'RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER'],
 ['RETRO SPOT MUG',
  'RETRO SPOT SUGAR JAM BOWL',
  'NEW ENGLAND CERAMIC CAKE SERVER',
  'RETRO SPOT TRADITIONAL TEAPOT']]

In [32]:
label_list[:2]

['RETRO SPOT TRADITIONAL TEAPOT', 'RETRO SPOT LARGE MILK JUG']

In [33]:
df_new = pd.DataFrame(columns=['Detail', 'label'])
df_new['Detail'] = df_list
df_new['label'] = label_list
df_new

,Detail,label
0,"[HAND OPEN SHAPE GOLD, RETRO SPOT MUG, RETRO S...",RETRO SPOT TRADITIONAL TEAPOT
1,"[RETRO SPOT MUG, RETRO SPOT SUGAR JAM BOWL, NE...",RETRO SPOT LARGE MILK JUG
2,"[MOROCCAN TEA GLASS, BLACK FLOWER CANDLE PLATE...",CERAMIC CAKE DESIGN SPOTTED PLATE
3,"[BLACK SILOUETTE CANDLE PLATE, VICTORIAN GLASS...",SAVE THE PLANET COTTON TOTE BAG
4,"[CERAMIC CAKE DESIGN SPOTTED PLATE, PINK CLEAR...",STRAWBERRY CERAMIC TRINKET BOX
...,...,...
74502,"[UNION STRIPE WITH FRINGE HAMMOCK, RED/CREAM ...",WHITE HANGING HEART T-LIGHT HOLDER
74503,"[OCEAN STRIPE HAMMOCK, BLUE/CREAM STRIPE FRING...",ENAMEL WATERING CAN CREAM
74504,"[WHITE HANGING HEART T-LIGHT HOLDER, BAG 250g ...",NATURAL SLATE HEART CHALKBOARD
74505,"[ENAMEL WATERING CAN CREAM, PORCELAIN HANGING ...",COLOUR GLASS T-LIGHT HOLDER HANGING


In [34]:
df_new['Detail'] = df_new['Detail'].apply(lambda x: '&&'.join(map(str, x)))
df_new

,Detail,label
0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...,RETRO SPOT TRADITIONAL TEAPOT
1,RETRO SPOT MUG&&RETRO SPOT SUGAR JAM BOWL&&NEW...,RETRO SPOT LARGE MILK JUG
2,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...,CERAMIC CAKE DESIGN SPOTTED PLATE
3,BLACK SILOUETTE CANDLE PLATE&&VICTORIAN GLASS ...,SAVE THE PLANET COTTON TOTE BAG
4,CERAMIC CAKE DESIGN SPOTTED PLATE&&PINK CLEAR ...,STRAWBERRY CERAMIC TRINKET BOX
...,...,...
74502,UNION STRIPE WITH FRINGE HAMMOCK&&RED/CREAM S...,WHITE HANGING HEART T-LIGHT HOLDER
74503,OCEAN STRIPE HAMMOCK&&BLUE/CREAM STRIPE FRINGE...,ENAMEL WATERING CAN CREAM
74504,WHITE HANGING HEART T-LIGHT HOLDER&&BAG 250g S...,NATURAL SLATE HEART CHALKBOARD
74505,ENAMEL WATERING CAN CREAM&&PORCELAIN HANGING B...,COLOUR GLASS T-LIGHT HOLDER HANGING


In [95]:
print(df_new.isnull().sum())
df_new.dropna(inplace=True)
df_new.isnull().sum()

Detail    0
label     0
dtype: int64


Detail    0
label     0
dtype: int64

In [96]:
print(df_new.duplicated().sum())
df_new.drop_duplicates(inplace=True)
df_new.reset_index(inplace=True)
df_new.pop('index')
df_new

0


,Detail,label
0,HAND OPEN SHAPE GOLD&&RETRO SPOT MUG&&RETRO SP...,RETRO SPOT TRADITIONAL TEAPOT
1,RETRO SPOT MUG&&RETRO SPOT SUGAR JAM BOWL&&NEW...,RETRO SPOT LARGE MILK JUG
2,MOROCCAN TEA GLASS&&BLACK FLOWER CANDLE PLATE&...,CERAMIC CAKE DESIGN SPOTTED PLATE
3,BLACK SILOUETTE CANDLE PLATE&&VICTORIAN GLASS ...,SAVE THE PLANET COTTON TOTE BAG
4,CERAMIC CAKE DESIGN SPOTTED PLATE&&PINK CLEAR ...,STRAWBERRY CERAMIC TRINKET BOX
...,...,...
71200,WOODEN SKITTLES GARDEN SET&&DOORMAT UNION FLAG...,WOODEN CROQUET GARDEN SET
71201,UNION STRIPE WITH FRINGE HAMMOCK&&RED/CREAM S...,WHITE HANGING HEART T-LIGHT HOLDER
71202,OCEAN STRIPE HAMMOCK&&BLUE/CREAM STRIPE FRINGE...,ENAMEL WATERING CAN CREAM
71203,WHITE HANGING HEART T-LIGHT HOLDER&&BAG 250g S...,NATURAL SLATE HEART CHALKBOARD


In [36]:
# df1 = df.copy()
# df1.pop('Detail')
# df1.drop_duplicates(inplace=True)
# df1.reset_index(inplace=True)
# df1.drop('index', axis=1, inplace=True)
# df1.drop_duplicates(inplace=True)
# df1.reset_index(inplace=True)
# df1.pop('index')
# display(df1.head())
# len(df1)

In [37]:
# df2 = pd.DataFrame(columns=['Invoice', 'Detail'])
# values = df['Invoice'].value_counts().index.sort_values()

# for i, v in enumerate(values):
#     df2.loc[i] = (v, '&&'.join(df[df['Invoice'] == v]['Detail']))
# df2

In [38]:
# df1_ = df1.drop(['Invoice'], axis=1)
# df1_

In [39]:
# df3 = pd.concat([df1_, df2], axis=1)
# df3

In [40]:
# token_df = df3.copy()
# token_df['Detail'] = token_df['Detail'].str.replace('&&', ' [SEP] ')
# sentences = token_df['Detail']
# sentences = ["[CLS] " + str(sentence).lower() + " [SEP]" for sentence in sentences]
# sentences = [sentence.replace('[sep]', '[SEP]') for sentence in sentences]
# sentences[0]

In [41]:
# token_df.head()

In [42]:
# sent = '[CLS] (2마리) 코다리매콤명태찜 [SEP] (小) 로스팅훈제족발+쟁반국수 [SEP] (2인분) 쫄볶이(치즈 오뎅 떡 계란2개 쫄면사리) [SEP] 세트 트러블슈프림양념치킨+후라이드치킨 메뉴 [SEP] B치즈닭발2인세트 +주먹밥+계란찜+샐러드+쿨피스'
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# tokenized_texts = [tokenizer.tokenize(sent)]
# tokenized_texts

In [43]:
# def BERTtokenizer(sentences):
#     # BERT의 토크나이저로 문장을 토큰으로 분리
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
#     # tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#     tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#     return tokenized_texts

In [44]:
# tokenized_texts = BERTtokenizer(sentences)

# print (sentences[0])
# print (tokenized_texts[0])
# len(tokenized_texts)

In [45]:
# def SPACEtokenizer(sentences):
#     tokenized_texts2 = [sent.split(' ') for sent in sentences]
#     return tokenized_texts2

In [46]:
# tokenized_texts2 = SPACEtokenizer(sentences)
# print(tokenized_texts2[0])
# len(tokenized_texts2)

In [47]:
# tokenized_df = token_df.copy()
# tokenized_df['Detail'] = tokenized_texts
# # print(len(tokenized_df))
# tokenized_df

In [48]:
# num = 0
# for i in range(len(tokenized_df['Detail'])):
#     num += len(tokenized_df['Detail'][i])
# print(num / len(tokenized_df))

# a, b = 0, 0
# for i in range(len(tokenized_df['Detail'])):
#     a = len(tokenized_df['Detail'][i])
#     if a > b:
#         b = a
# print(b)

In [49]:
# tokenized_df['Detail'] = tokenized_df['Detail'].apply(lambda x: ','.join(map(str, x)))
# tokenized_df.head()

In [50]:
# tokenized_df2 = token_df.copy()
# tokenized_df2['Detail'] = tokenized_texts2
# # print(len(tokenized_df2))
# tokenized_df2

In [51]:
# tokenized_df2['Detail'] = tokenized_df2['Detail'].apply(lambda x: ','.join(map(str, x)))
# tokenized_df2.head()

In [52]:
# Customer = df3['Customer'].unique()
# len(Customer)

In [53]:
# MAX_LEN = 512

In [54]:
# overlen = []
# tokenized_texts_copy = tokenized_texts.copy()
# for i in range(len(tokenized_texts_copy)):
#     if len(tokenized_texts_copy[i]) > MAX_LEN:
#         overlen.append(tokenized_texts_copy[i])
#         while 1:
#             id = tokenized_texts_copy[i].index('[SEP]')
#             tokenized_texts_copy[i] = tokenized_texts_copy[i][id:]
#             tokenized_texts_copy[i][0] = '[CLS]'
#             if len(tokenized_texts_copy[i]) <= MAX_LEN:
#                 break
# tok = []
# for i in range(len(tokenized_texts_copy)):
#     if len(tokenized_texts_copy[i]) > MAX_LEN:
#         tok.append(len(tokenized_texts_copy[i]))
# # print(max(tok))
# print(len(tok))
# print(len(tokenized_texts_copy))
# print(len(overlen[0]))
# print(len(overlen))
# print(overlen[0])

In [55]:
# overlen2 = []
# tokenized_texts_copy2 = tokenized_texts2.copy()
# for i in range(len(tokenized_texts_copy2)):
#     if len(tokenized_texts_copy2[i]) > MAX_LEN:
#         overlen2.append(tokenized_texts_copy2[i])
#         while 1:
#             id = tokenized_texts_copy2[i].index('[SEP]')
#             tokenized_texts_copy2[i] = tokenized_texts_copy2[i][id:]
#             tokenized_texts_copy2[i][0] = '[CLS]'
#             if len(tokenized_texts_copy2[i]) <= MAX_LEN:
#                 break
# tok = []
# for i in range(len(tokenized_texts_copy2)):
#     if len(tokenized_texts_copy2[i]) > MAX_LEN:
#         tok.append(len(tokenized_texts_copy2[i]))
# # print(max(tok))
# print(len(tok))
# print(len(tokenized_texts_copy2))
# print(len(overlen2[0]))
# print(len(overlen2))
# print(overlen2[0])

In [56]:
# idx = []
# for i in range(len(overlen)):
#     idx.append(tokenized_df.index[tokenized_df['Detail'] == ','.join(v for v in overlen[i])].tolist())
# print(len(idx))
# print(idx)

In [57]:
# idx2 = []
# for i in range(len(overlen2)):
#     idx2.append(tokenized_df2.index[tokenized_df2['Detail'] == ','.join(v for v in overlen2[i])].tolist())
# print(len(idx2))
# print(idx2)

In [58]:
# overlen_df = pd.DataFrame(columns=df3.columns)
# df_reduced = df3.copy()
# for i, v in enumerate(idx):
#     overlen_df.loc[i] = tokenized_df.loc[v[0]]
#     df_reduced.drop(v[0], inplace=True)
# overlen_df

In [59]:
# overlen_df2 = pd.DataFrame(columns=df3.columns)
# df_reduced2 = df3.copy()
# for i, v in enumerate(idx2):
#     overlen_df2.loc[i] = tokenized_df2.loc[v[0]]
#     df_reduced2.drop(v[0], inplace=True)
# overlen_df2

In [60]:
# df_reduced

In [61]:
# df_reduced2

In [62]:
# df_reduced = sortdf(df_reduced, ['Customer', 'Invoice'])
# df_reduced

In [63]:
# df_reduced2 = sortdf(df_reduced2, ['Customer', 'Invoice'])
# df_reduced2

In [64]:
# df_add = pd.DataFrame(columns=['Customer', 'Detail'])

# for i in range(len(df_reduced) - 1):
#     if df_reduced.loc[i, 'Customer'] != df_reduced.loc[i+1, 'Customer']:
#         continue
#     df_add.loc[i] = (df_reduced.loc[i, 'Customer'], df_reduced.loc[i, 'Detail'] + df_reduced.loc[i+1, 'Detail'])
# df_add

In [65]:
# df_add2 = pd.DataFrame(columns=['Customer', 'Detail'])

# for i in range(len(df_reduced2) - 1):
#     if df_reduced2.loc[i, 'Customer'] != df_reduced2.loc[i+1, 'Customer']:
#         continue
#     df_add2.loc[i] = (df_reduced2.loc[i, 'Customer'], df_reduced2.loc[i, 'Detail'] + df_reduced2.loc[i+1, 'Detail'])
# df_add2

In [66]:
# data_bert = pd.concat([df3.drop('Invoice', axis=1), df_add])
# data_bert.reset_index(inplace=True)
# data_bert.pop('index')
# data_bert

In [67]:
# data_space = pd.concat([df3.drop('Invoice', axis=1), df_add2])
# data_space.reset_index(inplace=True)
# data_space.pop('index')
# data_space

In [68]:
# # temp_bert = data_bert['Detail'].str.replace(' [SEP] ', '*')
# # type(temp_bert)
# series_bert = data_bert['Detail'].str.split('&&')
# print(type(series_bert))
# display(series_bert)
# print(series_bert[0][-1])
# len(series_bert[0])

In [69]:
# series_space = data_space['Detail'].str.split('&&')
# print(type(series_space))
# print(len(series_space))
# print(series_space[0][-1])
# len(series_space[0])

In [70]:
# label_bert = []

# for i in range(len(series_bert)):
#     l = series_bert[i][-1]
#     series_bert[i].pop()
#     label_bert.append(l)

# print(label_bert[:5])
# # series_bert[0][-1]
# len(label_bert)

In [71]:
# label_space = []

# for i in range(len(series_space)):
#     l = series_space[i][-1]
#     series_space[i].pop()
#     label_space.append(l)

# print(label_space[:5])
# len(label_space)

In [72]:
# data_bert['Detail'] = series_bert.apply(lambda x: '&&'.join(map(str, x)))
# data_bert.head()

In [73]:
# data_space['Detail'] = series_space.apply(lambda x: '&&'.join(map(str, x)))
# data_space.head()

In [74]:
# data_bert['label'] = label_bert
# data_space['label'] = label_space
# data_bert.dropna()
# data_space.dropna()

In [75]:
# print(sorted(data_bert['Detail'].unique())[0])
# sorted(data_space['Detail'].unique())[0]

In [76]:
# idx = data_bert[data_bert['Detail'] == ''].index
# data_bert.drop(idx, inplace=True)
# idx = data_space[data_space['Detail'] == ''].index
# data_space.drop(idx, inplace=True)

In [77]:
# data_bert.reset_index(inplace=True)
# data_bert.pop('index')
# data_space.reset_index(inplace=True)
# data_space.pop('index')

In [78]:
# token_data_bert = data_bert.copy()
# token_data_bert['Detail'] = token_data_bert['Detail'].str.replace('&&', ' [SEP] ')
# sentences_bert = token_data_bert['Detail']
# sentences_bert = ["[CLS] " + str(sentence).lower() + " [SEP]" for sentence in sentences_bert]
# sentences_bert = [sentence.replace('[sep]', '[SEP]') for sentence in sentences_bert]
# sentences_bert[0]

In [79]:
# token_data_space = data_space.copy()
# token_data_space['Detail'] = token_data_space['Detail'].str.replace('&&', ' [SEP] ')
# sentences_space = token_data_space['Detail']
# sentences_space = ["[CLS] " + str(sentence).lower() + " [SEP]" for sentence in sentences_space]
# sentences_space = [sentence.replace('[sep]', '[SEP]') for sentence in sentences_space]
# sentences_space[0]

In [80]:
# tokenized_texts_bert = BERTtokenizer(sentences_bert)

# print (sentences_bert[0])
# print (tokenized_texts_bert[0])
# len(tokenized_texts_bert)

In [81]:
# tokenized_texts_space = SPACEtokenizer(sentences_space)
# print(tokenized_texts_space[0])
# len(tokenized_texts_space)

In [82]:
# token_data_bert['Detail'] = tokenized_texts_bert
# token_data_space['Detail'] = tokenized_texts_space

In [83]:
# # df_add.reset_index(inplace=True)
# # df_add.pop('index')

# num = 0
# for i in range(7372, len(token_data_bert['Detail'])):
#     num += len(token_data_bert['Detail'][i])
# print(num / (len(token_data_bert) - 7372))

# a, b = 0, 0
# for i in range(len(token_data_bert['Detail'])):
#     a = len(token_data_bert['Detail'][i])
#     if a > b:
#         b = a
# print(b)

자르기

In [84]:
# # overlen = []
# tokenized_texts_bert_copy = tokenized_texts_bert.copy()
# for i in range(len(tokenized_texts_bert_copy)):
#     if len(tokenized_texts_bert_copy[i]) > MAX_LEN:
#         # overlen.append(tokenized_texts_bert_copy[i])
#         while 1:
#             id = tokenized_texts_bert_copy[i].index('[SEP]')
#             tokenized_texts_bert_copy[i] = tokenized_texts_bert_copy[i][id:]
#             tokenized_texts_bert_copy[i][0] = '[CLS]'
#             if len(tokenized_texts_bert_copy[i]) <= MAX_LEN:
#                 break
# tok = []
# for i in range(len(tokenized_texts_bert_copy)):
#     if len(tokenized_texts_bert_copy[i]) > MAX_LEN:
#         tok.append(len(tokenized_texts_bert_copy[i]))
# # print(max(tok))
# print(len(tok))
# print(len(tokenized_texts_bert_copy))
# # print(len(overlen[0]))
# # print(len(overlen))
# # print(overlen[0])

In [85]:
# # overlen = []
# tokenized_texts_space_copy = tokenized_texts_space.copy()
# for i in range(len(tokenized_texts_space_copy)):
#     if len(tokenized_texts_space_copy[i]) > MAX_LEN:
#         # overlen.append(tokenized_texts_space_copy[i])
#         while 1:
#             id = tokenized_texts_space_copy[i].index('[SEP]')
#             tokenized_texts_space_copy[i] = tokenized_texts_space_copy[i][id:]
#             tokenized_texts_space_copy[i][0] = '[CLS]'
#             if len(tokenized_texts_space_copy[i]) <= MAX_LEN:
#                 break
# tok = []
# for i in range(len(tokenized_texts_space_copy)):
#     if len(tokenized_texts_space_copy[i]) > MAX_LEN:
#         tok.append(len(tokenized_texts_space_copy[i]))
# # print(max(tok))
# print(len(tok))
# print(len(tokenized_texts_space_copy))
# # print(len(overlen[0]))
# # print(len(overlen))
# # print(overlen[0])

In [86]:
# token_data_bert['Detail'] = tokenized_texts_bert_copy
# token_data_space['Detail'] = tokenized_texts_space_copy

In [87]:
# # df_add.reset_index(inplace=True)
# # df_add.pop('index')

# num = 0
# for i in range(len(token_data_bert['Detail'])):
#     num += len(token_data_bert['Detail'][i])
# print(num / (len(token_data_bert)))

# a, b = 0, 0
# for i in range(len(token_data_bert['Detail'])):
#     a = len(token_data_bert['Detail'][i])
#     if a > b:
#         b = a
# print(b)

In [88]:
# 1token_data_bert['Detail'] = token_data_bert['Detail'].apply(lambda x: ','.join(map(str, x)))
# token_data_space['Detail'] = token_data_space['Detail'].apply(lambda x: ','.join(map(str, x)))

In [89]:
# idx = token_data_bert[token_data_bert['Detail'] == '[CLS],[SEP]'].index
# token_data_bert.drop(idx, inplace=True)
# idx = token_data_space[token_data_space['Detail'] == '[CLS],,[SEP]'].index
# token_data_space.drop(idx, inplace=True)

In [90]:
# token_data_bert.reset_index(inplace=True)
# token_data_bert.pop('index')
# token_data_space.reset_index(inplace=True)
# token_data_space.pop('index')

In [91]:
# token_data_bert

In [92]:
# len(token_data_bert['Detail'][11573])

In [93]:
# token_data_space

저장

In [94]:
# data_bert.to_csv("/content/drive/MyDrive/train_bert.csv", mode='w')
# data_space.to_csv("/content/drive/MyDrive/train_space.csv", mode='w')